In [0]:
import bz2
from collections import Counter
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import nltk
import torchtext
import numpy as np
import json
from torchtext import data
import spacy

In [0]:
## Mount drive in the left tab to store the dataset and model info
%cd /content/drive/'My Drive'/

In [0]:
train_file = bz2.BZ2File('./amazon_reviews/train.ft.txt.bz2')
test_file = bz2.BZ2File('./amazon_reviews/test.ft.txt.bz2')

In [0]:
train_file = train_file.readlines()
test_file = test_file.readlines()

In [0]:
print("Number of training reivews: " + str(len(train_file)))
print("Number of test reviews: " + str(len(test_file)))

Number of training reivews: 3600000
Number of test reviews: 400000


In [0]:
num_train = 100000 #We're training on the first 800,000 million reviews in the dataset
num_test = 20000 #Using 200,000 reviews from test set

train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]

In [0]:
train_labels = [0 if x.split()[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]

# Further Splitting it down between validation and training set
train_sentences, val_sentences = train_sentences[int(0.2 * len(train_sentences)) : ], train_sentences[: int(0.2 * len(train_sentences))]
train_labels, val_labels = train_labels[int(0.2 * len(train_labels)) :], train_labels[: int(0.2 * len(train_labels))]

In [0]:
print ("Size of training data, ", len(train_sentences))
print ("Size of validation data, ", len(val_sentences))
print ("Size of testing data, ", len(test_sentences))

Size of training data,  80000
Size of validation data,  20000
Size of testing data,  20000


In [0]:
del train_file, test_file

In [0]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [5]:
print (generate_bigrams(['this' , 'is', 'the', 'perfect', 'day', '!']))

['this', 'is', 'the', 'perfect', 'day', '!', 'is the', 'day !', 'perfect day', 'this is', 'the perfect']


In [0]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
fields = fields = [
    ('text', TEXT), 
    ('label', LABEL)
]

train_examples = [data.Example.fromlist([train_sentences[i], train_labels[i]], fields) 
                  for i in range(len(train_sentences))]
val_examples = [data.Example.fromlist([val_sentences[i], val_labels[i]], fields) 
                  for i in range(len(val_sentences))]
test_examples = [data.Example.fromlist([test_sentences[i], test_labels[i]], fields) 
                for i in range(len(test_sentences))]

In [0]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x : len(x.text),
    device = device)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx) 
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
        return self.fc(pooled)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [14]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.8858, -0.5027,  1.1485,  ...,  1.5460, -0.4453, -3.6734],
        [ 0.1901, -0.2545, -0.3214,  ...,  0.1637,  0.2881, -0.1090],
        [ 1.0519, -1.8847,  0.7483,  ..., -1.4481,  0.0162,  0.4964]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.291 | Train Acc: 89.56%
	 Val. Loss: 0.304 |  Val. Acc: 90.09%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.229 | Train Acc: 91.63%
	 Val. Loss: 0.310 |  Val. Acc: 90.89%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.198 | Train Acc: 92.87%
	 Val. Loss: 0.322 |  Val. Acc: 91.22%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.176 | Train Acc: 93.75%
	 Val. Loss: 0.335 |  Val. Acc: 91.45%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.160 | Train Acc: 94.39%
	 Val. Loss: 0.349 |  Val. Acc: 91.55%


In [0]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.304 | Test Acc: 90.19%
